<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Определение-стоимости-автомобилей" data-toc-modified-id="Определение-стоимости-автомобилей-1">Определение стоимости автомобилей</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-2">Подготовка данных</a></span></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-3">Обучение моделей</a></span></li><li><span><a href="#Анализ-моделей" data-toc-modified-id="Анализ-моделей-4">Анализ моделей</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-5">Тестирование модели</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-6">Вывод</a></span></li></ul></div>

# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В нашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Необходимо построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

# Подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
import warnings
warnings.filterwarnings('ignore')

In [2]:
try:
    data = pd.read_csv('C:/Users/datasets/autos.csv')
except:
    data = pd.read_csv('//datasets/autos.csv')


*Изучим общую информацию.*

In [3]:
data.head(10)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
5,2016-04-04 17:36:23,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,2016-04-04 00:00:00,0,33775,2016-04-06 19:17:07
6,2016-04-01 20:48:51,2200,convertible,2004,manual,109,2_reihe,150000,8,petrol,peugeot,no,2016-04-01 00:00:00,0,67112,2016-04-05 18:18:39
7,2016-03-21 18:54:38,0,sedan,1980,manual,50,other,40000,7,petrol,volkswagen,no,2016-03-21 00:00:00,0,19348,2016-03-25 16:47:58
8,2016-04-04 23:42:13,14500,bus,2014,manual,125,c_max,30000,8,petrol,ford,NaN,2016-04-04 00:00:00,0,94505,2016-04-04 23:42:13
9,2016-03-17 10:53:50,999,small,1998,manual,101,golf,150000,0,NaN,volkswagen,NaN,2016-03-17 00:00:00,0,27472,2016-03-31 17:17:06


In [4]:
data.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


*Проверим данные на наличие явных дубликатов.*

In [5]:
data.duplicated().sum()

4

*Удалим явные дубликты.*

In [6]:
data = data.drop_duplicates().reset_index(drop=True)

*Сохраним в новой переменной "data_filtered" информативные признаки.*

In [7]:
data_filtered = data[['Price', 'VehicleType', 'RegistrationYear', 'Gearbox', 'Power', 'Model',
                             'Kilometer', 'FuelType', 'Brand', 'Repaired']]

*Ознакомимся с количеством пропусков.*

In [8]:
data_filtered.isna().mean()


Price               0.000000
VehicleType         0.105795
RegistrationYear    0.000000
Gearbox             0.055968
Power               0.000000
Model               0.055607
Kilometer           0.000000
FuelType            0.092828
Brand               0.000000
Repaired            0.200793
dtype: float64

*Рассмотрим значения 'Gearbox'.*

In [9]:
data_filtered['Gearbox'].value_counts()

manual    268249
auto       66283
Name: Gearbox, dtype: int64

*Заменим пропуски в "Gearbox" на "manual".*

In [10]:
data_filtered['Gearbox'] = data_filtered['Gearbox'].fillna('manual')

*Рассмотрим значения 'Repaired'.*

In [11]:
data_filtered['Repaired'].unique()

array([nan, 'yes', 'no'], dtype=object)

*Логично предположить,что пользователь не указал наличие ремонта из-за его отсутствия.Заменим пропуски на "no".*

In [12]:
data_filtered['Repaired'] = data_filtered['Repaired'].fillna('no')

*Рассмотрим значения "Model".*

In [13]:
data_filtered['Model'].unique()

array(['golf', nan, 'grand', 'fabia', '3er', '2_reihe', 'other', 'c_max',
       '3_reihe', 'passat', 'navara', 'ka', 'polo', 'twingo', 'a_klasse',
       'scirocco', '5er', 'meriva', 'arosa', 'c4', 'civic', 'transporter',
       'punto', 'e_klasse', 'clio', 'kadett', 'kangoo', 'corsa', 'one',
       'fortwo', '1er', 'b_klasse', 'signum', 'astra', 'a8', 'jetta',
       'fiesta', 'c_klasse', 'micra', 'vito', 'sprinter', '156', 'escort',
       'forester', 'xc_reihe', 'scenic', 'a4', 'a1', 'insignia', 'combo',
       'focus', 'tt', 'a6', 'jazz', 'omega', 'slk', '7er', '80', '147',
       '100', 'z_reihe', 'sportage', 'sorento', 'v40', 'ibiza', 'mustang',
       'eos', 'touran', 'getz', 'a3', 'almera', 'megane', 'lupo', 'r19',
       'zafira', 'caddy', 'mondeo', 'cordoba', 'colt', 'impreza',
       'vectra', 'berlingo', 'tiguan', 'i_reihe', 'espace', 'sharan',
       '6_reihe', 'panda', 'up', 'seicento', 'ceed', '5_reihe', 'yeti',
       'octavia', 'mii', 'rx_reihe', '6er', 'modus', 'fox'

*По причине отсутствия возможностей логично заменить пропуски, поставим на их место заглушки "unknown".*

In [14]:
data_filtered['Model'] = data_filtered['Model'].fillna('unknown')

*Рассмотрим значения "VehicleType".*

In [15]:
data_filtered['VehicleType'].unique()

array([nan, 'coupe', 'suv', 'small', 'sedan', 'convertible', 'bus',
       'wagon', 'other'], dtype=object)

In [16]:
data_filtered['VehicleType'].value_counts()

sedan          91457
small          79830
wagon          65165
bus            28775
convertible    20203
coupe          16161
suv            11996
other           3288
Name: VehicleType, dtype: int64

*Заполним пропуски самым популярным кузовом для каждой модели.*

In [17]:
dictionary_vehicle_type = data_filtered.groupby('Model')['VehicleType'].agg(lambda x:x.value_counts().index[0]).to_dict()

In [18]:
def vehicle_type_function(row):    
    if pd.isnull(row['VehicleType']):
        return dictionary_vehicle_type[row['Model']]
    else:
        return row['VehicleType']

data_filtered['VehicleType'] = data_filtered.apply(vehicle_type_function, axis=1) 

*Заполним пропуски самым популярным типом топлива для каждой модели.*

In [19]:
dictionary_fuel_type = data_filtered.groupby('Model')['FuelType'].agg(lambda x:x.value_counts().index[0]).to_dict()

In [20]:
def fuel_type_function(row):    
    if pd.isnull(row['FuelType']):
        return dictionary_fuel_type[row['Model']]
    else:
        return row['FuelType']

data_filtered['FuelType'] = data_filtered.apply(fuel_type_function, axis=1)

*Рассмотрим значения целевого признака 'Price'.*

In [21]:
data_filtered['Price'].value_counts()

0        10772
500       5670
1500      5394
1000      4648
1200      4594
         ...  
13180        1
10879        1
2683         1
634          1
8188         1
Name: Price, Length: 3731, dtype: int64

*Избавимся от нулевых значений.*

In [22]:
data_filtered = data_filtered.query('Price>0')

*Рассмотрим значения признака 'RegistrationYear'.*

In [23]:
data_filtered['RegistrationYear'].unique()

array([1993, 2011, 2004, 2001, 2008, 1995, 2014, 1998, 2005, 1910, 2016,
       2007, 2009, 2002, 2018, 1997, 1990, 2017, 1981, 2003, 1994, 1991,
       1984, 2006, 1999, 2012, 2010, 2000, 1992, 2013, 1996, 1985, 1989,
       2015, 1982, 1976, 1983, 1973, 1969, 1971, 1987, 1986, 1988, 1980,
       1970, 1965, 1945, 1925, 1974, 1979, 1955, 1978, 1972, 1968, 1977,
       1961, 1966, 1975, 1963, 1964, 1960, 5000, 1958, 1967, 1959, 1956,
       3200, 1000, 1941, 9999, 8888, 1500, 2200, 4100, 1962, 1929, 1957,
       1940, 3000, 2066, 1949, 2019, 1937, 1951, 1800, 1953, 1954, 1234,
       8000, 5300, 9000, 2900, 6000, 5900, 5911, 1400, 1950, 4000, 1948,
       1952, 8500, 1932, 1255, 3700, 3800, 4800, 1942, 7000, 1935, 1933,
       1936, 6500, 1923, 2290, 1930, 1001, 9450, 1944, 2500, 1943, 1934,
       1938, 1928, 5555, 5600, 1600, 1111, 2222, 1039, 1300, 2800, 1931,
       4500, 1602, 7800, 1947, 1927, 7100, 8200, 1946])

*Мы видим некорректные значения. Машина не может быть зарегистрирована позже текущего года. Удалим данные значения.*

In [24]:
data_filtered = data_filtered.query('RegistrationYear<2024')

*Рассмотрим значения признака 'Power'.*

In [25]:
data_filtered['Power'].unique()

array([    0,   190,   163,    75,    69,   102,   109,   125,   101,
         105,   140,   115,   131,    60,   136,   160,   231,    90,
          50,   118,   193,    99,   113,   218,   122,   129,    70,
         306,    95,    61,   177,    80,   170,    55,   143,    64,
         286,   232,   150,   156,    82,   155,    54,   185,    87,
         180,    86,    84,   224,   235,   200,   178,   265,    77,
         110,   144,   120,   116,   184,   126,   204,    88,   194,
         305,   197,   179,   250,    45,   313,    41,   165,    98,
         130,   114,   211,    56,   201,   213,    58,   107,    83,
         174,   100,   220,    85,    73,   192,    68,    66,   299,
          74,    52,   147,    65,   310,    71,    97,   239,   203,
           5,   300,   103,   258,   320,    63,    81,   148,    44,
         145,   280,   260,   104,   188,   333,   186,   117,   141,
          59,   132,   234,   158,    39,    92,    51,   135,   230,
          53,   209,

*У транспортного средства не может быть 0 лошадиных сил.Верхнюю границу определим в районе 1000 лошадиных сил.*

In [26]:
data_filtered = data_filtered.query('0<Power<1000')

# Обучение моделей

*Обучать будем 3 модели LinearRegression, CatBoost и LightGBM.*

**LinearRegression**

Воспользуемся OHE,т.к.этот метод лучше всего подходит для линейной регрессии. 

In [27]:
ohe = pd.get_dummies(data_filtered, drop_first=True)

Разделим данные на тренировочные и тестовые выборки.

In [28]:
features_train_lr, features_test_lr, target_train_lr, target_test_lr = train_test_split(
    ohe.drop('Price', axis=1), data_filtered.Price, test_size=0.25, random_state=12345)

Масштабируем количественные признаки.

In [29]:
numeric = ['RegistrationYear', 'Power', 'Kilometer']

scaler = StandardScaler()
scaler.fit(features_train_lr[numeric])

features_train_lr[numeric] = scaler.transform(features_train_lr[numeric])
features_test_lr[numeric] = scaler.transform(features_test_lr[numeric])
features_train_lr.head()

,RegistrationYear,Power,Kilometer,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,VehicleType_small,VehicleType_suv,VehicleType_wagon,...,Brand_skoda,Brand_smart,Brand_sonstige_autos,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,Repaired_yes
117476,-0.037941,0.782005,0.586830,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
96367,-1.872300,-1.195379,0.586830,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
213297,-0.431018,-1.213688,-0.094017,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
239755,1.010264,0.360895,-2.408894,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
15443,-0.431018,-0.188378,0.586830,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0


*Обучим модель и измерим время обучения и предсказания, рассчитаем RMSE.*

In [30]:
mse_lr = cross_val_score(LinearRegression(), features_train_lr, target_train_lr, cv=5, scoring='neg_mean_squared_error')
print('RMSE:',(mse_lr.mean()* -1)**0.5)

RMSE: 2760.8069265611425


In [31]:
%%time
lr_model = LinearRegression()
lr_model.fit(features_train_lr, target_train_lr)

CPU times: user 14.6 s, sys: 4.41 s, total: 19 s
Wall time: 18.9 s


LinearRegression()

In [32]:
%%time
lr_prediction = lr_model.predict(features_train_lr)

CPU times: user 208 ms, sys: 146 ms, total: 354 ms
Wall time: 312 ms


**CatBoost**

*Разделим данные на тренировочные и тестовые выборки и выделим категориальные признаки.*

In [33]:
cat_features = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired']

features_train_cat, features_test_cat, target_train_cat, target_test_cat = train_test_split(
    data_filtered.drop('Price', axis=1), data_filtered.Price, test_size=0.25, random_state=12345)

*Найдём лучшие гиперпараметры используя GridSearchCV.*

In [34]:
params = {
    'learning_rate':[0.1, 0.5],
    'depth':[2, 10],
    'l2_leaf_reg': [1, 9]
    
}

grid = GridSearchCV(CatBoostRegressor(loss_function="RMSE", iterations=200, silent=True),
                    param_grid=params,
                    cv=5,
                    n_jobs=-1,    
                    scoring='neg_mean_squared_error',
                    verbose=False)

grid.fit(features_train_cat, target_train_cat, cat_features=cat_features)
print('RMSE: {} при значениях гиперпараметров: {}'.format((-grid.best_score_)**0.5, grid.best_params_))

RMSE: 1610.8213953629738 при значениях гиперпараметров: {'depth': 10, 'l2_leaf_reg': 9, 'learning_rate': 0.5}


*Обучим модель c наилучшими параметрами и измерим время обучения, предсказания и RMSE.*

In [35]:
%%time
cat_model = CatBoostRegressor(loss_function="RMSE", iterations=200, 
                              learning_rate=0.5, depth=10, l2_leaf_reg=9, silent=True)

cat_model.fit(features_train_cat, target_train_cat, cat_features=cat_features)

CPU times: user 1min 7s, sys: 322 ms, total: 1min 7s
Wall time: 1min 10s


In [36]:
%%time
cat_prediction = cat_model.predict(features_train_cat)

CPU times: user 861 ms, sys: 7.91 ms, total: 869 ms
Wall time: 873 ms


**LightGBM**

*Возьмём данные Catboost, но изменим тип категориальных данных на "category".*

In [37]:
features_train_cat[cat_features] = features_train_cat[cat_features].astype('category')
features_test_cat[cat_features] = features_test_cat[cat_features].astype('category')
features_train_cat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 230188 entries, 117476 to 251243
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   VehicleType       230188 non-null  category
 1   RegistrationYear  230188 non-null  int64   
 2   Gearbox           230188 non-null  category
 3   Power             230188 non-null  int64   
 4   Model             230188 non-null  category
 5   Kilometer         230188 non-null  int64   
 6   FuelType          230188 non-null  category
 7   Brand             230188 non-null  category
 8   Repaired          230188 non-null  category
dtypes: category(6), int64(3)
memory usage: 8.6 MB


In [38]:
params = {
    'learning_rate': [0.1, 0.5],
    'max_depth': [3, 9],
    'num_leaves': [30, 90, 10],
    
}

grid = GridSearchCV(LGBMRegressor(random_state=12345, num_iterations=200),
                    param_grid=params,
                    cv=5,
                    n_jobs=-1,
                    scoring='neg_mean_squared_error',
                    verbose=False)

grid.fit(features_train_cat, target_train_cat, categorical_feature=cat_features)

GridSearchCV(cv=5,
             estimator=LGBMRegressor(num_iterations=200, random_state=12345),
             n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.5], 'max_depth': [3, 9],
                         'num_leaves': [30, 90, 10]},
             scoring='neg_mean_squared_error', verbose=False)

In [39]:
print('RMSE: {} при значениях гиперпараметров: {}'.format((-grid.best_score_)**0.5, grid.best_params_))

RMSE: 1586.8177859573923 при значениях гиперпараметров: {'learning_rate': 0.1, 'max_depth': 9, 'num_leaves': 90}


*Обучим модель c наилучшими параметрами и измерим время обучения, предсказания и RMSE.*

In [40]:
%%time
lgbm_model = LGBMRegressor(random_state=12345, num_iterations=200, min_data_in_leaf=50, 
                           learning_rate=0.1, max_depth=9, num_leaves=90, boost='gbdt')

lgbm_model.fit(features_train_cat, target_train_cat, categorical_feature=cat_features)

CPU times: user 18.4 s, sys: 111 ms, total: 18.6 s
Wall time: 18.6 s


LGBMRegressor(boost='gbdt', max_depth=9, min_data_in_leaf=50,
              num_iterations=200, num_leaves=90, random_state=12345)

In [41]:
%%time
prediction_lgbm = lgbm_model.predict(features_train_cat)

CPU times: user 6.64 s, sys: 0 ns, total: 6.64 s
Wall time: 6.56 s


# Анализ моделей

*Таблица с результатами скорости обучения, предсказания и RMSE.*

In [42]:
data = [['18.9 s', '312 ms', 2760.80],
        ['1min 10s', '873 ms', 1610.82],
        ['18.6 s', '6.56 s', 1586.81]]
models = ['LinearRegression', 'CatBoost', 'LightGBM']

pd.DataFrame(data=data, index=models, columns=['fit_time', 'predict_time', 'RMSE'])

,fit_time,predict_time,RMSE
LinearRegression,18.9 s,312 ms,2760.80
CatBoost,1min 10s,873 ms,1610.82
LightGBM,18.6 s,6.56 s,1586.81


В таблице мы можем наблюдать, что у LinearRegression существенно хуже значение RMSE - 2760.80 в сравнении с другими моделями, поэтому сравнивать будем CatBoost и LGBM. У CatBoost и LGBM значения RMSE очень близки(CatBoost - 1610.82 , LGBM - 1586.81), но чуть лучше у LightGBM - 1586.81. По скорости обучения лидирует модель LGBM у неё время 18.6 s, у CatBoost 1min 10s. В скорости предсказания лучше оказалась модель CatBoost - 873 ms у LGBM - 6.56 s. Поскольку заказчику важны скорость обучения, скорость предсказания и значение RMSE, для проверки на тестовой выборке мы возьмём модель LightGBM, т.к. она оказалась лучше в двух показателях.

# Тестирование модели

In [43]:
%%time

lgbm_model = LGBMRegressor(random_state=12345, num_iterations=200, min_data_in_leaf=50, 
                           learning_rate=0.1, max_depth=9, num_leaves=90, boost='gbdt')

lgbm_model.fit(features_train_cat, target_train_cat, categorical_feature=cat_features)

[LightGBM] [Warning] boosting is set with boosting_type=gbdt, will be overridden by boost=gbdt. Current value: boosting=gbdt
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
CPU times: user 19.8 s, sys: 106 ms, total: 19.9 s
Wall time: 20 s


LGBMRegressor(boost='gbdt', max_depth=9, min_data_in_leaf=50,
              num_iterations=200, num_leaves=90, random_state=12345)

In [44]:
%%time
prediction_lgbm = lgbm_model.predict(features_test_cat)

CPU times: user 2.18 s, sys: 1.27 ms, total: 2.18 s
Wall time: 2.12 s


In [45]:
print('RMSE :', mean_squared_error(target_test_cat, prediction_lgbm) ** 0.5)

RMSE : 1595.2568412460757


# Вывод

Обучив модель LightGBM на тестовой выборке мы получили следующие значения:

- Время обучения: 20 s
- Время предсказания: 2.12 s
- RMSE: 1595.2568412460757